In [1]:
import ee
import geopandas as gpd
import geemap
import xarray as xr
import wxee

In [2]:
ee.Authenticate()

True

In [ ]:
import ee
import geemap
import wxee
import xarray as xr
import time
import os
from tqdm import tqdm
import json
from concurrent.futures import ThreadPoolExecutor, TimeoutError

ee.Initialize()

CLASS_NAMES = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
CLASS_NUMBER = ee.List.sequence(0, 8)

def get_buenos_aires_feature():
    provinces = ee.FeatureCollection("FAO/GAUL/2015/level1")
    exclude_provinces = ['Tierra Del Fuego', 'Santa Cruz', 'Neuquen', 'Chubut', 'Rio Negro']
    exclude_filter = ee.Filter.Not(ee.Filter.inList('ADM1_NAME', exclude_provinces))
    argentina_provinces = provinces.filter(ee.Filter.eq('ADM0_NAME', 'Argentina'))
    filtered_provinces = argentina_provinces.filter(exclude_filter)
    return filtered_provinces

buenos_aires_feature = get_buenos_aires_feature()
Argentina_geometry = buenos_aires_feature.geometry()

# Create fishnet directly
print("Creating fishnet...")
fishnet = geemap.fishnet(Argentina_geometry, rows=300, cols=120)
print("Fishnet created.")

def compute_dominant_class(image):
    filtered_image = image.select(CLASS_NAMES)
    max_prob = filtered_image.reduce(ee.Reducer.max())
    most_prob_class = max_prob.eq(filtered_image)
    most_prob_class = most_prob_class.multiply(ee.Image.constant(CLASS_NUMBER))
    dominant_class = most_prob_class.reduce(ee.Reducer.sum()).rename('dominant_class')
    return dominant_class.uint8().copyProperties(image, ['system:time_start'])

def ensure_time_property(image):
    time_start = image.get('system:time_start')
    if time_start is None:
        year = image.get('year')
        if year is not None:
            time_start = ee.Date.fromYMD(year, 1, 1).millis()
            return image.set('system:time_start', time_start)
    if time_start is None:
        default_time = ee.Date('2000-01-01').millis()
        return image.set('system:time_start', default_time)
    return image

def process_grid_cell(i, aoi):
    print(f"Processing grid cell {i}")
    try:
        dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2014", "2026").filterBounds(aoi)
        def clip_image(image):
            return image.clip(aoi).copyProperties(image, ['system:time_start'])
        dw_filtered = dw_filtered.map(clip_image)
        yearly_ts = dw_filtered.aggregate_time(
            frequency="year",
            reducer=ee.Reducer.mean()
        )
        yearly_ts = yearly_ts.map(ensure_time_property)
        yearly_dominant_class = yearly_ts.map(compute_dominant_class)

        ds = yearly_dominant_class.wx.to_xarray(region=aoi, scale=10)

        ds = ds.fillna(-1)
        ds = ds.clip(min=0, max=8)
        ds["dominant_class"] = ds["dominant_class"].astype("uint8")
        ds["x"] = ds["x"].astype("float32")
        ds["y"] = ds["y"].astype("float32")
        ds["time"] = ds["time"].dt.strftime("%Y")

        output_dir = "D:\\Factor-Data\\Tesina\\Tesina\\Definitivo"
        os.makedirs(output_dir, exist_ok=True)

        # Add compression settings
        encoding = {"dominant_class": {"zlib": True, "complevel": 5}}

        ds.to_netcdf(os.path.join(output_dir, f"dominant_class_array_{i}.nc"), encoding=encoding)

        print(f"Completed processing grid cell {i}")
        return True
    except Exception as e:
        print(f"Error processing grid cell {i}: {str(e)}")
        return False

def process_with_timeout(i, aoi, timeout=1200):  # 20 minutes timeout
    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(process_grid_cell, i, aoi)
        try:
            return future.result(timeout=timeout)
        except TimeoutError:
            print(f"Timeout occurred for grid cell {i}")
            return False
        except Exception as e:
            print(f"Error processing grid cell {i}: {str(e)}")
            return False

def load_progress(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return {'last_processed': -1, 'failed_cells': []}

def save_progress(filename, progress):
    with open(filename, 'w') as f:
        json.dump(progress, f)

def main():
    batch_size = 5
    total_cells = fishnet.size().getInfo()
    progress_file = 'processing_progress.json'
    progress = load_progress(progress_file)

    pbar = tqdm(total=total_cells, initial=progress['last_processed'] + 1, desc="Processing grid cells")

    try:
        for batch_start in range(progress['last_processed'] + 1, total_cells, batch_size):
            batch_end = min(batch_start + batch_size, total_cells)
            batch_success_count = 0

            for i in range(batch_start, batch_end):
                if i in progress['failed_cells']:
                    continue
                aoi = ee.Feature(fishnet.toList(fishnet.size()).get(i).getInfo()).geometry()
                success = process_with_timeout(i, aoi)
                if success:
                    batch_success_count += 1
                    progress['last_processed'] = i
                else:
                    progress['failed_cells'].append(i)
                save_progress(progress_file, progress)

            pbar.update(batch_success_count)
            tqdm.write(f"Completed batch {batch_start}-{batch_end-1}")
            time.sleep(60)  # Wait for 60 seconds between batches

    except Exception as e:
        tqdm.write(f"An error occurred in the main loop: {str(e)}")
    finally:
        pbar.close()

    tqdm.write("Processing complete!")
    if progress['failed_cells']:
        tqdm.write(f"Failed cells: {progress['failed_cells']}")

if __name__ == "__main__":
    main()

Creating fishnet...
Fishnet created.


Processing grid cells:   1%|          | 180/21410 [00:00<?, ?it/s]

Processing grid cell 182


Requesting data:   0%|          | 0/10 [00:00<?, ?it/s]

Error processing grid cell 182: 400 Client Error: Bad Request for url: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/6c21760c8467d6267a999fc8f0be3626-54b835d052453e3457851ba44d1e87c0:getPixels
Processing grid cell 183


...

Completed processing grid cell 259
Completed batch 255-259
Processing grid cell 260


Requesting data:   0%|          | 0/10 [00:00<?, ?it/s]

Completed processing grid cell 260
Processing grid cell 261


Requesting data:   0%|          | 0/10 [00:00<?, ?it/s]

Completed processing grid cell 261
